In [ ]:
using Pkg; Pkg.activate(joinpath(@__DIR__, ".."))
using LinearAlgebra
using HybridRobotDynamics
using HiLQR

In [ ]:
"""
Quaternion Utils
"""

const Hquat = [zeros(1, 3); I(3)]

function skew(a::Vector)
    @assert size(a) == (3,)
    return [
        0.0 -a[3] a[2];
        a[3] 0.0 -a[1];
        -a[2] a[1] 0.0
    ]
end

function L_or_R(q::Vector, is_L::Bool)::Matrix
    @assert size(q) == (4,)
    s = q[1]
    v = q[2:4]
    sign = is_L ? 1 : -1
    return [s -v'; v s*I(3) + sign*skew(v)]
end

function Lquat(q::Vector)::Matrix
    return L_or_R(q, true)
end

function Rquat(q::Vector)::Matrix
    return L_or_R(q, false)
end

function Gquat(q::Vector)::Matrix
    return Lquat(q) * Hquat
end

function Qquat(q::Vector)::Matrix
    return Hquat' * Rquat(q)' * Lquat(q) * Hquat
end

function random_unit_quat()::Vector{Float64}
    unscaled_quat = randn(4)
    return unscaled_quat / norm(unscaled_quat)
end

nothing

In [ ]:
"""
13-State Quadrotor Model
"""

function get_quadrotor_model(
    gravity::Float64,
    mass::Float64,
    principal_moments::Vector{Float64},
    torque_consts::Vector{Float64},
    rotor_xs::Vector{Float64},
    rotor_ys::Vector{Float64},
)::HybridSystem
    @assert size(principal_moments) == (3,)
    @assert size(torque_consts) == (4,)
    @assert size(rotor_xs) == (4,)
    @assert size(rotor_ys) == (4,)

    # Constants
    g = [0.0, 0.0, -gravity]
    J = diagm(principal_moments)

    # Control allocation
    K = [zeros(2,4); ones(1,4)]
    B = [
        rotor_ys';
        -rotor_xs';
        (torque_consts .* [-1.0, 1.0, -1.0, 1.0])'
    ]

    function quadrotor_flow(x::Vector, u::Vector)::Vector
        p = x[1:3]
        q = x[4:7]
        v = x[8:10]
        ω = x[11:13]
        return [
            Qquat(q) * v;
            0.5 * Gquat(q) * ω;
            Qquat(q)'*g + K*u/mass - cross(ω, v);
            J \ (B*u - cross(ω, J*ω))
        ]
    end

    # Return HybridSystem
    nx = 13
    nu = 4
    mI = HybridMode(quadrotor_flow)
    modes = Dict(:flight => mI)
    transitions = Dict(Tuple{Symbol, Transition}[])
    return HybridSystem(nx, nu, transitions, modes)
end

nothing

In [ ]:
"""
Solver Setup
"""

# Crazyflie 2.1 parameters: https://arxiv.org/pdf/1608.05786
g = 9.81
m = 0.027
Jdiag = [1.436e-5, 1.395e-5,  2.173e-5]
b = 7.9379e-12 / 3.1582e-10 * ones(4)
c = 0.0283 * ones(4)
d = 0.0283 * ones(4)
system = get_quadrotor_model(g, m, Jdiag, b, c, d)

# Stage and terminal costs
Q = 1e-6 * diagm([2e1*ones(3); 1e1*ones(4); ones(3); ones(3)])
R = 1e-6 * I(system.nu)
Qf = 1e3 * Q
stage(x, u) = x'*Q*x + u'*R*u
terminal(x) = x'*Qf*x

# RK4 integrator
rk4 = ExplicitIntegrator(:rk4)

# Problem parameters
N = 50
Δt = 0.02
params = HiLQR.Parameters(system, stage, terminal, rk4, N, Δt)

# Reference trajectory
pref = [2.0, 2.0, 2.0]
qref = [1.0; zeros(3)]
xref = [pref; qref; zeros(6)]
uref = zeros(system.nu)
params.xrefs = [xref for k = 1:N]
params.urefs = [uref for k = 1:(N-1)]

# Initial conditions
p0 = zeros(3)
q0 = qref #random_unit_quat()
v0 = zeros(3)
ω0 = zeros(3)
params.x0 = [p0; q0; v0; ω0]
params.mI = :flight

nothing

In [ ]:
"""
Solve using HiLQR
"""

# Warm-start
warmstart = true
sol = HiLQR.Solution(params)
if warmstart
    pxs = Vector(range(p0[1], pref[1], N))
    pys = Vector(range(p0[2], pref[2], N))
    pzs = Vector(range(p0[3], pref[3], N))
    for k = 1:N
        sol.xs[k] = [pxs[k]; pys[k]; pzs[k]; zeros(10)]
    end
end

# Solve
cache = HiLQR.Cache(params)
@time HiLQR.solve!(sol, cache, params; multishoot=true, max_iter=100)

# Visualize states
xs = reduce(vcat, sol.xs)
plot_2d_states(
    N, system.nx, (1,3), xs;
    xlim=(-2,4), ylim=(-2,4),
    xlabel="x", ylabel="z"
)

nothing